In [4]:
from pyspark.sql import SparkSession

In [11]:
from pyspark.sql import SparkSession

def patch_pyspark_accumulators():
    from inspect import getsource
    import pyspark.accumulators as pa
    exec(getsource(pa._start_update_server).replace("localhost", "127.0.0.1"), pa.__dict__)

patch_pyspark_accumulators()
spark = SparkSession.builder.getOrCreate()
# sparkSession [Errno 8] nodename nor servname provided, or not known 가 발생하게 된다면 해당 코드 작성 
# vpn을 사용하거나 했을 때 localhost 주소가 달라져서 그런 것 같음 

In [12]:
spark = SparkSession.builder.master("local").appName("learn-sql").getOrCreate()

In [13]:
stocks = [
    ('Google', 'GOOGL', 'USA', 2984, 'USD'), 
    ('Netflix', 'NFLX', 'USA', 645, 'USD'),
    ('Amazon', 'AMZN', 'USA', 3518, 'USD'),
    ('Tesla', 'TSLA', 'USA', 1222, 'USD'),
    ('Tencent', '0700', 'Hong Kong', 483, 'HKD'),
    ('Toyota', '7203', 'Japan', 2006, 'JPY'),
    ('Samsung', '005930', 'Korea', 70600, 'KRW'),
    ('Kakao', '035720', 'Korea', 125000, 'KRW'),
]

In [14]:
# 스키마 지정 
stockSchema = ["name", "ticker", "country", "price", "currency"]

In [15]:
# DataFrame 생성 
df = spark.createDataFrame(data=stocks, schema = stockSchema)
# data와 schema 지정 

In [16]:
print(df.dtypes)

[('name', 'string'), ('ticker', 'string'), ('country', 'string'), ('price', 'bigint'), ('currency', 'string')]


In [17]:
print(df.show())

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USA|  1222|     USD|
|Tencent|  0700|Hong Kong|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+

None


In [18]:
# spark sql 실행 
df.createOrReplaceTempView("stocks")
# df를 temporary View로 등록 

In [20]:
spark.sql("select name from stocks").show()
# spark로 sql을 실행할 수 있음

+-------+
|   name|
+-------+
| Google|
|Netflix|
| Amazon|
|  Tesla|
|Tencent|
| Toyota|
|Samsung|
|  Kakao|
+-------+



In [21]:
spark.sql("select name, price from stocks").show()

+-------+------+
|   name| price|
+-------+------+
| Google|  2984|
|Netflix|   645|
| Amazon|  3518|
|  Tesla|  1222|
|Tencent|   483|
| Toyota|  2006|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [22]:
spark.sql("select name, price from stocks where country = 'Korea'").show()
# 조건문 지정 가능 (해당 주식의 나라가 Korea인 것만 )

+-------+------+
|   name| price|
+-------+------+
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [23]:
spark.sql("select name, price from stocks where price > 2000").show()
# 조건문 지정 가능 (해당 주식의 가격이 2000 초과인 것만 )

+-------+------+
|   name| price|
+-------+------+
| Google|  2984|
| Amazon|  3518|
| Toyota|  2006|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [24]:
spark.sql("select name, price from stocks where price > 2000 and country = 'USA'").show()
# 조건문 지정 가능 (해당 주식의 가격이 2000 초과인 것과 나라가 USA인 것만 )

+------+-----+
|  name|price|
+------+-----+
|Google| 2984|
|Amazon| 3518|
+------+-----+



In [26]:
spark.sql("select name, price from stocks where country like 'U%'").show()
# 나라 이름이 U로 시작하는 모든 주식의 이름과 가격 

+-------+-----+
|   name|price|
+-------+-----+
| Google| 2984|
|Netflix|  645|
| Amazon| 3518|
|  Tesla| 1222|
+-------+-----+



In [28]:
spark.sql("select name, price from stocks where country like 'U%' and name not like '%e%'").show()
# 나라 이름이 U로 시작하고 주식의 이름에 e가 들어가지 않는 모든 주식의 이름과 가격 

+------+-----+
|  name|price|
+------+-----+
|Amazon| 3518|
+------+-----+



In [29]:
spark.sql("select name, price from stocks where price between 1000 and 10000").show()
# 가격이 1000에서 10000사이의 모든 주식의 이름과 가격 

+------+-----+
|  name|price|
+------+-----+
|Google| 2984|
|Amazon| 3518|
| Tesla| 1222|
|Toyota| 2006|
+------+-----+



In [30]:
spark.sql("select name, price from stocks where country = 'USA'").show()

+-------+-----+
|   name|price|
+-------+-----+
| Google| 2984|
|Netflix|  645|
| Amazon| 3518|
|  Tesla| 1222|
+-------+-----+



In [33]:
spark.sql("select name, price, currency from stocks where currency = 'USD' and price > (select price from stocks where name='Tesla')").show()
# 서브쿼리도 사용할 수 있음 
# 테슬라보다 주식의 가격이 높으면서 currency가 USD인 주식의 이름, 가격, 화폐


+------+-----+--------+
|  name|price|currency|
+------+-----+--------+
|Google| 2984|     USD|
|Amazon| 3518|     USD|
+------+-----+--------+



In [34]:
spark.sql("select name, price from stocks order by price asc").show()
# order by를 사용해서 정렬 가능 

+-------+------+
|   name| price|
+-------+------+
|Tencent|   483|
|Netflix|   645|
|  Tesla|  1222|
| Toyota|  2006|
| Google|  2984|
| Amazon|  3518|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [35]:
spark.sql("select name, price from stocks order by length(name)").show()


+-------+------+
|   name| price|
+-------+------+
|  Tesla|  1222|
|  Kakao|125000|
| Amazon|  3518|
| Toyota|  2006|
| Google|  2984|
|Netflix|   645|
|Samsung| 70600|
|Tencent|   483|
+-------+------+



In [36]:
spark.sql("select sum(price) from stocks where country ='Korea'").show()
# 한국에 있는 주식의 가격의 합 

+----------+
|sum(price)|
+----------+
|    195600|
+----------+



In [37]:
spark.sql("select mean(price) from stocks where country ='Korea'").show()

+-----------+
|mean(price)|
+-----------+
|    97800.0|
+-----------+



In [38]:
spark.sql("select count(price) from stocks where country ='Korea'").show()

+------------+
|count(price)|
+------------+
|           2|
+------------+



In [39]:
spark.sql("select count(price) from stocks where country in ('Korea', 'USA')").show()
# 한국와 미국의 주식의 개수

+------------+
|count(price)|
+------------+
|           6|
+------------+



## JOIN

In [40]:
earnings = [
    ('Google', 27.99, 'USD'), 
    ('Netflix', 2.56, 'USD'),
    ('Amazon', 6.12, 'USD'),
    ('Tesla', 1.86, 'USD'),
    ('Tencent', 11.01, 'HKD'),
    ('Toyota', 224.82, 'JPY'),
    ('Samsung', 1780., 'KRW'),
    ('Kakao', 705., 'KRW')
]
# 해당 회사가 주당 얼마를 버는지 나타내는 것


In [41]:
from pyspark.sql.types import StringType, FloatType, StructType, StructField

In [43]:
earningsSchema = StructType([
    StructField("name", StringType(), True),
    StructField("eps", FloatType(), True),
    StructField("currency", StringType(), True)
])

In [48]:
earningDF = spark.createDataFrame(data=earnings, schema=earningsSchema)
# DataFrame으로 변환

In [49]:
earningDF.dtypes

[('name', 'string'), ('eps', 'float'), ('currency', 'string')]

In [50]:
# dataframe을 sql에 사용하기 위해서 temporaryView에 등록해야 SQL 가능 
earningDF.createOrReplaceTempView("earnings")

In [51]:
earningDF.select("*").show()

+-------+------+--------+
|   name|   eps|currency|
+-------+------+--------+
| Google| 27.99|     USD|
|Netflix|  2.56|     USD|
| Amazon|  6.12|     USD|
|  Tesla|  1.86|     USD|
|Tencent| 11.01|     HKD|
| Toyota|224.82|     JPY|
|Samsung|1780.0|     KRW|
|  Kakao| 705.0|     KRW|
+-------+------+--------+



In [54]:
spark.sql("select * from stocks join earnings on stocks.name = earnings.name").show()
# tempview에 등록한 이름이 table명임으로 join을 위처럼 할 수 있음 
# stocks.name과 earnings.name이 같은 값들만 가져옴 

+-------+------+---------+------+--------+-------+------+--------+
|   name|ticker|  country| price|currency|   name|   eps|currency|
+-------+------+---------+------+--------+-------+------+--------+
| Amazon|  AMZN|      USA|  3518|     USD| Amazon|  6.12|     USD|
| Google| GOOGL|      USA|  2984|     USD| Google| 27.99|     USD|
|  Kakao|035720|    Korea|125000|     KRW|  Kakao| 705.0|     KRW|
|Netflix|  NFLX|      USA|   645|     USD|Netflix|  2.56|     USD|
|Samsung|005930|    Korea| 70600|     KRW|Samsung|1780.0|     KRW|
|Tencent|  0700|Hong Kong|   483|     HKD|Tencent| 11.01|     HKD|
|  Tesla|  TSLA|      USA|  1222|     USD|  Tesla|  1.86|     USD|
| Toyota|  7203|    Japan|  2006|     JPY| Toyota|224.82|     JPY|
+-------+------+---------+------+--------+-------+------+--------+



In [56]:
# PER(주가 수익 비율) : Price / EPS

spark.sql("select stocks.name, (stocks.price/earnings.eps) from stocks join earnings on stocks.name = earnings.name").show()
# 연산을 수행할 수 있음 

+-------+------------------+
|   name|     (price / eps)|
+-------+------------------+
| Amazon| 574.8366120563447|
| Google| 106.6095042658442|
|  Kakao| 177.3049645390071|
|Netflix| 251.9531306315913|
|Samsung|39.662921348314605|
|Tencent| 43.86920889728746|
|  Tesla|  656.989242258975|
| Toyota| 8.922693419839167|
+-------+------------------+

